# Multi-Agent Workflows

Creating **multiple agents with different roles** in **LangGraph** involves:

1. Defining **distinct agents** using `ChatOpenAI` or any LLM.
2. Assigning **tools or behaviors** specific to each agent.
3. Connecting them in a **LangGraph** using **nodes and transitions**.



### ✅ Example: Two Agents – *Math Expert* and *Text Expert*

```python
from langchain.agents import Tool
from langchain.tools import tool
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import create_tool_calling_executor

# Step 1: Define tools
@tool
def multiply(a: int, b: int) -> int:
    """Multiplies two integers."""
    return a * b

@tool
def reverse_string(s: str) -> str:
    """Reverses a string."""
    return s[::-1]

# Step 2: Define agents
math_agent = create_tool_calling_executor(
    llm=ChatOpenAI(model="gpt-3.5-turbo-0125"),
    tools=[multiply],
    name="MathExpert"
)

text_agent = create_tool_calling_executor(
    llm=ChatOpenAI(model="gpt-3.5-turbo-0125"),
    tools=[reverse_string],
    name="TextExpert"
)

# Step 3: Define shared state
class AgentState(dict):
    """Custom dict to act as state."""
    pass

# Step 4: Define LangGraph
builder = StateGraph(AgentState)

# Add agent nodes
builder.add_node("math_agent", math_agent)
builder.add_node("text_agent", text_agent)

# Step 5: Add transitions
def router(state: AgentState):
    question = state["input"]
    if any(char.isdigit() for char in question):
        return "math_agent"
    else:
        return "text_agent"

builder.set_entry_point(router)
builder.add_edge("math_agent", END)
builder.add_edge("text_agent", END)

graph = builder.compile()

# Step 6: Run
print("🔢 Math task:")
result_math = graph.invoke({"input": "What is 6*9?"})
print(result_math)

print("\n🔤 Text task:")
result_text = graph.invoke({"input": "Reverse the word 'LangGraph'"})
print(result_text)
```



### 💡 Output Example:

```
🔢 Math task:
{'input': 'What is 6*9?', 'math_agent': 'The result of 6 multiplied by 9 is 54.'}

🔤 Text task:
{'input': "Reverse the word 'LangGraph'", 'text_agent': "The reverse of 'LangGraph' is 'hparGgnaL'."}
```



### 🧠 Concepts Used:

* **Two separate agents**, each tied to specific tools.
* **Routing logic** based on input content.
* **Modular nodes** using LangGraph’s state and transitions.



# Agent collaboration using message-passing

### 🤖 Agent Collaboration using **Message-Passing** in LangGraph

LangGraph enables **multiple agents to collaborate** by **passing messages** (like a chat thread) between each other, mimicking multi-agent communication and coordination workflows.



## 🔁 What is Message Passing?

**Message passing** is when each agent operates on a **shared message state** and appends or updates messages (e.g., chat-style) that are passed along the LangGraph. It enables:

* Turn-taking between agents
* Context-aware responses
* Emergent reasoning via collaboration



## ✅ Example: Two Agents Collaborating via Message Passing

### 🔧 Setup: Two roles – *Planner* and *Executor*

```python
from langchain_core.messages import HumanMessage, AIMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END

# Define state
class MessageState(dict):
    pass

# Create LLMs for planner and executor agents
planner_llm = ChatOpenAI(model="gpt-4")
executor_llm = ChatOpenAI(model="gpt-4")

# Define planner node
def planner_node(state: MessageState):
    messages = state.get("messages", [])
    messages.append(HumanMessage(content="What steps should be taken to solve this problem?"))
    response = planner_llm.invoke(messages)
    messages.append(response)
    return {"messages": messages}

# Define executor node
def executor_node(state: MessageState):
    messages = state["messages"]
    messages.append(HumanMessage(content="Now execute the planned steps."))
    response = executor_llm.invoke(messages)
    messages.append(response)
    return {"messages": messages}

# Build the graph
builder = StateGraph(MessageState)
builder.add_node("planner", planner_node)
builder.add_node("executor", executor_node)

# Define message-passing transitions
builder.set_entry_point("planner")
builder.add_edge("planner", "executor")
builder.add_edge("executor", END)

# Compile and run
graph = builder.compile()

# Start the conversation
initial_state = {"messages": [HumanMessage(content="Build a house")]}
final_state = graph.invoke(initial_state)

# View result
for msg in final_state["messages"]:
    role = "🧑" if isinstance(msg, HumanMessage) else "🤖"
    print(f"{role}: {msg.content}")
```



### 🧠 Output Sample

```
🧑: Build a house
🧑: What steps should be taken to solve this problem?
🤖: 1. Find location 2. Get permits 3. Lay foundation ...
🧑: Now execute the planned steps.
🤖: I will begin by choosing a location and applying for permits...
```



## 🧩 Benefits of Message Passing in LangGraph

| Benefit                 | Description                                                      |
| ----------------------- | ---------------------------------------------------------------- |
| 🧠 Multi-turn reasoning | Agents can build context from previous steps                     |
| 🔁 Reusable state       | Messages act as shared memory across nodes                       |
| 👥 Collaboration        | Agents can hand off tasks or escalate based on results           |
| 🧱 Modular structure    | Each agent is isolated yet communicates via a simple message API |






## 🎭 What Are Role-Playing Agents?

Role-playing agents are LLM-based agents that **mimic real-world roles** (e.g., software developer, project planner, QA engineer) by using tailored system prompts, tool access, and task-specific logic.



## 🧱 Common Roles

| Agent       | Purpose                                    |
| ----------- | ------------------------------------------ |
| 🧠 Planner  | Break down user goals into clear steps     |
| 💻 Coder    | Write actual code based on the plan        |
| ✅ Reviewer  | Check and improve code quality             |
| 📦 Executor | Run or simulate the output of written code |



## 🧠 Step-by-Step Example (Planner → Coder → Reviewer)

### 🔧 Setup Requirements

```bash
pip install langgraph langchain-openai langchain
```



### 🧬 Code

```python
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, AIMessage
from langchain_openai import ChatOpenAI

# Shared state
class AgentState(dict):
    pass

# Define agents with system prompts
planner = ChatOpenAI(model="gpt-4", temperature=0.5).bind(system="You are a software planner. Break the user request into steps.")
coder = ChatOpenAI(model="gpt-4", temperature=0.3).bind(system="You are a Python coder. Write Python code based on instructions.")
reviewer = ChatOpenAI(model="gpt-4", temperature=0.3).bind(system="You are a code reviewer. Improve code quality and fix bugs.")

# Define LangGraph nodes
def planner_node(state: AgentState):
    msgs = state["messages"]
    response = planner.invoke(msgs)
    msgs.append(response)
    return {"messages": msgs}

def coder_node(state: AgentState):
    msgs = state["messages"]
    response = coder.invoke(msgs)
    msgs.append(response)
    return {"messages": msgs}

def reviewer_node(state: AgentState):
    msgs = state["messages"]
    response = reviewer.invoke(msgs)
    msgs.append(response)
    return {"messages": msgs}

# Build LangGraph
builder = StateGraph(AgentState)
builder.add_node("planner", planner_node)
builder.add_node("coder", coder_node)
builder.add_node("reviewer", reviewer_node)

builder.set_entry_point("planner")
builder.add_edge("planner", "coder")
builder.add_edge("coder", "reviewer")
builder.add_edge("reviewer", END)

graph = builder.compile()

# Initial input
state = {"messages": [HumanMessage(content="Build a Python function to sort a list of numbers.")]}

# Run the graph
final_state = graph.invoke(state)

# Print each message
for msg in final_state["messages"]:
    role = "🧑" if isinstance(msg, HumanMessage) else "🤖"
    print(f"{role}: {msg.content}")
```



## 🧪 Sample Output

```
🧑: Build a Python function to sort a list of numbers.
🤖 (Planner): 1. Define function. 2. Use sorted(). 3. Return result.
🤖 (Coder): def sort_numbers(nums): return sorted(nums)
🤖 (Reviewer): Code looks good. Added type hints and docstring.
```



## ✅ Tips

* Use **`system` prompts** to enforce role behavior
* You can add a **memory module** (like ConversationBufferMemory) to simulate context retention
* Add **conditional routing** for error handling or multi-turn collaboration



## 🎯 Goal

Let’s simulate a **multi-agent team** of AI agents that:

> 🔧 *"Builds a Python function to scrape all links from a webpage."*



## 🧠 Agent Roles

| Agent         | Role Description                                        |
| ------------- | ------------------------------------------------------- |
| 🧠 Planner    | Breaks down the task into subtasks                      |
| 🔍 Researcher | Searches for info if needed (e.g., how to scrape links) |
| 💻 Coder      | Writes Python code                                      |
| ✅ Reviewer    | Reviews code and suggests improvements                  |
| 🧪 Tester     | Simulates testing and provides output                   |



## 🧰 Requirements

Install dependencies:

```bash
pip install langgraph langchain-openai langchain
```



## 🧬 Full Code

```python
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI

# 💾 Shared state container
class AgentState(dict):
    pass

# 🔌 Agents with system prompts
planner = ChatOpenAI(model="gpt-4", temperature=0.5).bind(
    system="You are a project planner. Break the user request into development steps."
)

researcher = ChatOpenAI(model="gpt-4", temperature=0.3).bind(
    system="You are a researcher. Provide technical background info to help the coder."
)

coder = ChatOpenAI(model="gpt-4", temperature=0.3).bind(
    system="You are a Python programmer. Write complete code for the task."
)

reviewer = ChatOpenAI(model="gpt-4", temperature=0.3).bind(
    system="You are a senior software reviewer. Refactor and improve the code."
)

tester = ChatOpenAI(model="gpt-4", temperature=0.3).bind(
    system="You are a code tester. Simulate executing the code and describe the result."
)

# 🧱 LangGraph nodes
def planner_node(state: AgentState):
    response = planner.invoke(state["messages"])
    state["messages"].append(response)
    return {"messages": state["messages"]}

def researcher_node(state: AgentState):
    response = researcher.invoke(state["messages"])
    state["messages"].append(response)
    return {"messages": state["messages"]}

def coder_node(state: AgentState):
    response = coder.invoke(state["messages"])
    state["messages"].append(response)
    return {"messages": state["messages"]}

def reviewer_node(state: AgentState):
    response = reviewer.invoke(state["messages"])
    state["messages"].append(response)
    return {"messages": state["messages"]}

def tester_node(state: AgentState):
    response = tester.invoke(state["messages"])
    state["messages"].append(response)
    return {"messages": state["messages"]}

# 🧩 Build LangGraph
builder = StateGraph(AgentState)

builder.add_node("planner", planner_node)
builder.add_node("researcher", researcher_node)
builder.add_node("coder", coder_node)
builder.add_node("reviewer", reviewer_node)
builder.add_node("tester", tester_node)

builder.set_entry_point("planner")
builder.add_edge("planner", "researcher")
builder.add_edge("researcher", "coder")
builder.add_edge("coder", "reviewer")
builder.add_edge("reviewer", "tester")
builder.add_edge("tester", END)

graph = builder.compile()

# 🚀 Initial user query
initial_state = {"messages": [HumanMessage(content="Create a Python function to scrape all links from a webpage.")]}

# ✅ Run the graph
final_state = graph.invoke(initial_state)

# 📤 Output results
print("\n📜 Final Agent Messages:\n")
for msg in final_state["messages"]:
    role = "🧑" if isinstance(msg, HumanMessage) else "🤖"
    print(f"{role}: {msg.content}\n")
```



## 🧪 Sample Output (Condensed)

```
🧑: Create a Python function to scrape all links from a webpage.

🤖 (Planner): 1. Research method to scrape links. 2. Write code. 3. Review. 4. Test.

🤖 (Researcher): Use requests + BeautifulSoup to fetch and parse links.

🤖 (Coder): def extract_links(url): ... [Python code using BeautifulSoup]

🤖 (Reviewer): Added error handling and docstring.

🤖 (Tester): Code tested with example.com — works fine and prints all links.
```



## ✅ Key Concepts Demonstrated

* Agent memory through `messages`
* Multi-role collaboration using LangGraph flow
* Deterministic function execution (graph control)
* OpenAI's GPT agents as role-players



